In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd
import string
import re
import numpy as np
from collections import Counter
import unicodedata
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import spacy
from gensim.utils import simple_preprocess
from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support, accuracy_score
from sklearn.model_selection import StratifiedKFold
from collections import defaultdict
import pickle
import json
path = "/content/drive/My Drive/cs 583/sentiment analysis/"

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
%%capture
!pip install transformers
!pip install evaluate
!pip install datasets #restart

In [ ]:
# from datasets import load_dataset
from transformers import AutoTokenizer
# from transformers import Trainer
# from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
from datasets import load_metric
from scipy.special import softmax
import csv
import urllib.request

## To reformat input/output for demo

In [ ]:
i= 3#substitute with best split
MODEL = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(f'{path}model/Roberta_Finetuned/Roberta_Finetuned_{i}')

Downloading:   0%|          | 0.00/747 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [ ]:
def predict(text):
  encoded_input = tokenizer(text, return_tensors='pt')
  output = model(**encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
  
  ranking = np.argsort(scores)
  ranking = ranking[::-1]
  l = ranking[0]
  # l= labels[ranking[0]]
  if l == 0: return -1
  elif l == 1:return 0
  else: return 1

In [ ]:
#Load and clean input
filename = 'obama' #should be obama & romney
# for filename in filenames:
df = pd.read_csv(f'{path}data/test/{filename}.csv')
df = df[['tweet_id', 'tweet']]
df.fillna('', inplace = True)

def preprocess(example):
  new_text = [] 
  for t in example.split(" "):
      t = '@user' if t.startswith('@') and len(t) > 1 else t
      t = 'http' if t.startswith('http') else t
      new_text.append(t)
  new_text = " ".join(new_text)
  return new_text

df['tweet'] = df.tweet.apply(preprocess)

x_test = df.tweet
# y_test = df.sentiment #comment during demo as true labels not present in test file

df['pred'] = x_test.apply(predict)
#Save in required format in output folder
df['tweet_id']=df['tweet_id'].astype(str)+';;'
df['tweet_id']=df['tweet_id']+df['pred'].astype(str)
df= df[['tweet_id']]
np.savetxt(f'{path}output/demo/{filename}.txt', df.values, fmt='%s')


In [ ]:
df.to_csv(path+"output/demo/"+filename+".csv")
print(f"{filename} predictions saved")

romney predictions saved
